In [ ]:
import torch
from torch.utils.data import Dataset,DataLoader #常用工具区utils .data数据区
import pandas as pd
import numpy as np

torch.__version__#tourch版本

torch.cuda.is_available()#有没有显卡，有没有被激活 [如果为false，gpu是否支持cuda]

dir(torch.cuda.is_available())#打开某个package具有的功能并展示

help(torch.cuda.is_available)#解释包的每个功能的用法

In [ ]:
#不支持一边用cpu、另一边同gpu计算
a = torch.ones(3,3)
b = torch.ones(3,3)

a = a.to('cpu')
b = b.to('cuda')
#a+b 两个在不同设备上运行（cpu、gpu）的代码不能被计算出来

import torch
a = torch.Tensor([[1,2,3],[1,2,3]])
b = torch.unsqueeze(a,0)#扩充，反之压缩
print(a,b)

from PIL import image #打开图片
#如果只用展示某个固定的图片
img = Image.open("../data/plch2/bobby.jpg")

img.show()#展示图品

#多个图片的时候，分开写
img_path = "../data/plch2/bobby.jpg"
img = Image.open(img_path)

In [1]:
from torchvision import models
#torchvision 是PyTorch中专门用来处理图像的库

dir(models)#查看有哪些模型可用

#创建模型实例
alexnet = models.AlexNet()
alexnet#此时，alexnet是一个可以运行AlexNet架构的对象

resnet = models.resnet101(pretrained=True)
resnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

在colab上如何加载数据集：
1.数据集在github：  
!git clone https://github.com/Bakytnur16/Mydata.git

2.查看是否已经加载到目录中：  
!ls-R

# 数据预处理

In [ ]:
from torchvision import transforms
#预处理：变换图像
preprocess = transforms.Compose([
    transforms.Resize(256),#将图片像素更改为256*256
    transforms.CenterCrop(224),#围绕中心将图像裁剪为224×224个像素
    transforms.ToTensor(),#转换为张量
    transforms.Normalize( #对RGB分量进行归一化处理，有固定的均值和标准值
        mean=[0.485,0.456,0.406],
        std=[0.229, 0.224, 0.225]
    )])

In [ ]:
img = Image.open("...jpg")
img_t = preprocess(img) #预处理管道传递图像

import torch
batch_t = torch.unsqueeze(img_t,0)#unsqueeze对数据维度进行扩充，反之压缩
#在img_t的0位置加上一个维度为1的维度
#按照网络期望的方式对输入的张量进行重塑、裁剪、归一化处理


# 图像识别 【resnet】


In [ ]:
#1. 创建resnet实例 -> 引入模型
from torchvision import models #torchvision 是PyTorch中专门用来处理图像的库
resnet = models.resnet101(pretrained=True) #预训练为正，已经通过一批训练数据后的模型

#2. 数据预处理
from torchvision import transforms
preprocess=(transforms.Compose([
    transforms.Resize(256),#调整图片像素
    transforms.CenterCrop(224),#围绕圆心剪切图片大小
    transforms.ToTensor(),#转换为张量
    transforms.Normalize(#设置颜色，使其大致相同【同一个滤镜】
        mean = [0.485,0.456,0.406],
        std = [0.229,0.224,0.225]
    )]))

#3. 预处理输入【测试】数据
from PIL import Image
img = Image.open("/content/Mydata/hymenoptera_data/train/ants/0013035.jpg")
img_t = preprocess(img) #通过预处理通道后，即可转换为张量

import torch
batch_t = torch.unsqueeze(img_t,0) #均一化，加标签列？

# 在新数据上运行训练过的模型的过程被称为推理（inference），测试模型过程叫推理

#4. 评估数据
resnet.eval() #评估模式：模型内部的权值不会改变
#如果不启动评估模式，那么我们导入的预训练模型，批归一化、dropout就没意义了

out = resnet(batch_t) #输出为带有1000个标签的分数

#5. 输出标签，找出最高分的标签

#加载带有1000个标签的文件夹
with open('/content/dlwpt-code/data/p1ch2/imagenet_classes.txt') as f:
  labels = [line.strip() for line in f.readlines()]

_, index = torch.max(out, 1)

#没有_,返回values和index，有回复310
#labels[index]返回当下索引的字符 -> ant

percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100 
#torch.nn.functional.softmax()将输出归一化到[0,1]后除以总和，数字转换为百分比

labels[index[0]], percentage[index[0]].item() 


#输出前五个类别
_, indices = torch.sort(out, descending=True) 
[(labels[idx]) for idx in indices[0][:5]]

_, indices = torch.sort(out, descending=True) 
[(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]
#输出百分比
